Paper: [https://arxiv.org/pdf/2210.03629](https://arxiv.org/pdf/2210.03629)


In [ ]:
from utils import load_api_key
API_KEY = load_api_key()

# Implementación "from scratch" 

In [ ]:
import os
import re
import math
import json
from openai import OpenAI

API_KEY = API_KEY if API_KEY else os.environ["OPENAI_API_KEY"]

OPENAI_CLIENT = OpenAI(
    api_key=API_KEY,
)

OPENAI_MODEL = "gpt-4o-mini"

In [ ]:
def sum_two_elements(a: int, b: int) -> int:
    """
    Computes the sum of two integers.

    Args:
        a (int): The first integer to be summed.
        b (int): The second integer to be summed.

    Returns:
        int: The sum of `a` and `b`."
    """
    return a + b

def multiply_two_elements(a: int, b: int) -> int:
    """
    Computes the product of two integers.

    Args:
        a (int): The first integer to be multiplied.
        b (int): The second integer to be multiplied.

    Returns:
        int: The product of `a` and `b`."
    """
    return a * b

def compute_log(x: int) -> float:
    """
    Computes the natural logarithm of a number.

    Args:
        x (int): The number to compute the logarithm of.

    Returns:
        float: The natural logarithm of `x`."
    """
    if x <= 0:
        raise ValueError("The input must be a positive number.")

    return math.log(x)



Ahora definiremos un System Prompt que vamos a emplear en todas las llamadas a la API de OpenAI. Fijaos que dentro de las etiquetas de `<tools></tools>` hemos introducido las definiciones de las funciones. Esto es exactamente igual a lo que hará LangChain si creamos nuestra propia Tool o si usamos algunas de las Tools de `langchain_community`.

Además, en la sesión de ejemplo, hemos definido una iteración del bucle de ReAct:

pensamiento -> acción (tool) -> observación -> pensamiento -> acción (tool) -> observación -> pensamiento ...

In [ ]:
SYSTEM_PROMPT = """Eres un modelo de IA que llama a funciones. Operas ejecutando un ciclo con los siguientes pasos:

1. Pensamiento
2. Acción
3. Observación

Se te proporcionan definiciones de funciones dentro de etiquetas XML `<tools></tools>`.  
Puedes llamar a una o más funciones para ayudar con la consulta del usuario. 
No hagas suposiciones sobre qué valores debes introducir en las funciones.

Para cada llamada de función, devuelve un objeto JSON con el nombre de la función y los argumentos dentro de las etiquetas XML `<tool_call></tool_call>` como sigue:

<tool_call> {"name": <function-name>, "arguments": <args-dict>, "id": <monotonically-increasing-id>} </tool_call>

Aquí están las herramientas / acciones disponibles:
 
<tools>
{"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n\n    Args:\n    a (int): The first integer to be summed.\n    b (int): The second integer to be summed.\n\n    Returns:\n    int: The sum of `a` and `b`.\n"}
{"name": "multiply_two_elements", "description": "\n    Multiplies two integers.\n\n    Args:\n    a (int): The first integer.\n    b (int): The second integer.\n\n    Returns:\n    int: The product of `a` and `b`.\n"}
{"name": "compute_log", "description": "\n    Computes the logarithm of an integer `x` with an optional base.\n    Args:\n    x (int): The number to compute the logarithm for.\n    base (int, optional): The logarithm base, default is `e`.\n    Returns:\n    float: The logarithm of `x` in the given base.\n"}
</tools>

Ejemplo de sesión:

<question>¿Cuál es la temperatura actual en Madrid?</question>
<thought>Necesito obtener el clima actual en Madrid</thought>
<tool_call>{"name": "get_current_weather", "arguments": {"location": "Madrid", "unit": "celsius"}, "id": 0}</tool_call>

Te llamarán nuevamente con esto:

<observation>{0: {"temperature": 25, "unit": "celsius"}}</observation>

Entonces debes producir la salida:

<response>La temperatura actual en Madrid es de 25 grados Celsius</response>

Restricciones adicionales:

Si el usuario te pregunta algo que no esté relacionado con ninguna de las herramientas anteriores, 
responde libremente encerrando tu respuesta con etiquetas <response></response>.
"""

Las restricciones adicionales son importantes para que no intente ejecutar las tools en todas las iteraciones del bucle.

In [ ]:
USER_QUESTION = """Quiero calcular la suma de 1234 y 5678 y multiplicar el resultado por 5.
Luego quiero calcular el logaritmo natural de ese resultado."""

In [ ]:
chat_history = [
    {
        "role": "system",
        "content": SYSTEM_PROMPT
    },
    {
        "role": "user",
        "content": f"<question>{USER_QUESTION}</question>"
    }
]

In [ ]:
output = OPENAI_CLIENT.chat.completions.create(
    model=OPENAI_MODEL,
    messages=chat_history
).choices[0].message.content

print(output)

In [ ]:
chat_history.append({
    "role": "assistant",
    "content": output
})

In [ ]:
def extract_tag_content(text: str, tag: str) -> str:
    """
    Extracts the content of a tag from a text.

    Args:
        text (str): The text to extract the tag content from.
        tag (str): The tag to extract the content from.

    Returns:
        str: The content of the tag.
    """
    tag_pattern = f"<{tag}>(.*?)</{tag}>"
    matched_content = re.findall(tag_pattern, text, re.DOTALL)
    return [json.loads(content) for content in matched_content]

In [ ]:
extract_tag_content(output, tag="tool_call")[0]

In [ ]:
arguments = extract_tag_content(output, tag="tool_call")[0]["arguments"]
tool_result = sum_two_elements(**arguments)
tool_result

In [ ]:
chat_history.append({
    "role": "user",
    "content": f"<observation>{tool_result}</observation>"
})

In [ ]:
chat_history

#### Bucle 2

In [ ]:
output = OPENAI_CLIENT.chat.completions.create(
    model=OPENAI_MODEL,
    messages=chat_history
).choices[0].message.content

print(output)

In [ ]:
chat_history.append({
    "role": "assistant",
    "content": output
})

In [ ]:
arguments = extract_tag_content(output, tag="tool_call")[0]["arguments"]
tool_result = multiply_two_elements(**arguments)
tool_result

In [ ]:
chat_history.append({
    "role": "user",
    "content": f"<observation>{tool_result}</observation>"
})
chat_history

#### Bucle 3

In [ ]:
output = OPENAI_CLIENT.chat.completions.create(
    model=OPENAI_MODEL,
    messages=chat_history
).choices[0].message.content

print(output)

In [ ]:
chat_history.append({
    "role": "assistant",
    "content": output
})

In [ ]:
arguments = extract_tag_content(output, tag="tool_call")[0]["arguments"]
tool_result = compute_log(**arguments)
tool_result

In [ ]:
chat_history.append({
    "role": "user",
    "content": f"<observation>{tool_result}</observation>"
})
chat_history

#### Bucle 4

In [ ]:
output = OPENAI_CLIENT.chat.completions.create(
    model=OPENAI_MODEL,
    messages=chat_history
).choices[0].message.content

print(output)

Si queréis ver como quedaría un codigo más profesional, podéis ver el siguiente ejemplo: [Otro ejemplo de implementación de ReAct](https://medium.com/the-ai-forum/create-a-react-agent-from-scratch-without-using-any-llm-frameworks-only-with-python-and-groq-c10510d32dbc)

# ReAct Agent con LangGraph

## Ejemplo 1 con LangGraph

In [ ]:
from langchain_core.tools import tool
import math

@tool
def sum_two_elements(a: int, b: int) -> int:
    """
    Computes the sum of two integers.

    Args:
        a (int): The first integer to be summed.
        b (int): The second integer to be summed.

    Returns:
        int: The sum of `a` and `b`."
    """
    return a + b

@tool
def multiply_two_elements(a: int, b: int) -> int:
    """
    Computes the product of two integers.

    Args:
        a (int): The first integer to be multiplied.
        b (int): The second integer to be multiplied.

    Returns:
        int: The product of `a` and `b`."
    """
    return a * b

@tool
def compute_log(x: int) -> float:
    """
    Computes the natural logarithm of a number.

    Args:
        x (int): The number to compute the logarithm of.

    Returns:
        float: The natural logarithm of `x`."
    """
    if x <= 0:
        raise ValueError("The input must be a positive number.")

    return math.log(x)

tools = [sum_two_elements, multiply_two_elements, compute_log]

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(llm, tools)


In [ ]:
response = agent_executor.invoke(
    {"messages": """Quiero calcular la suma de 1234 y 5678 y multiplicar el resultado por 5. Luego quiero calcular el logaritmo natural de ese resultado."""},
)

msgs = [msg.pretty_print() for msg in response["messages"]]

## Ejemplo 2 con LangGraph

### Preparar las tools para nuestros agentes

In [ ]:
# Tavily tool
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_search = TavilySearchResults(max_results=3)
tools = [tavily_search]

# Try API without an LLM
search_results = tavily_search.invoke("¿Qué temperatura hace en Madrid hoy?")
display(search_results)

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(
    lang="es", 
    top_k_results=2,
    # doc_content_chars_max=1000
)
wikipedia_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
tools = [tavily_search, wikipedia_tool]
# tools = [wikipedia_tool]

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
llm_with_tools = llm.bind_tools(tools)

response = llm_with_tools.invoke("Hola!")

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
response = llm_with_tools.invoke("¿Qué temperatura hace en Madrid?")

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

# Todavía no estamos llamando a la Tool, solo nos está diciendo que lo hagamos. Para llamarlo, necesitamos crear nuestro agente.

#### Create the ReAct Agent

In [ ]:
#!pip install langgraph -q

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

# We use the model, not the model_with_tools, because the agent will handle the tool calls for us.
memory = MemorySaver()
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abcd1234"}}

In [ ]:
agent_executor

In [ ]:
response = agent_executor.invoke(
    {"messages": "Hola!"},
    config
)

response["messages"]

In [ ]:
response = agent_executor.invoke(
    {"messages": "¿Quién es el actual presidente de Portugal?"},
    config
)

msgs = [msg.pretty_print() for msg in response["messages"]]

In [ ]:
response = agent_executor.invoke(
    {"messages": "Busca en internet la fecha y hora de ahora mismo en Madrid."},
    config
)

msgs = [msg.pretty_print() for msg in response["messages"]]

In [ ]:
response = agent_executor.invoke(
    {"messages": "¿Va a llover mañana en Madrid?"},
    config
)

msgs = [msg.pretty_print() for msg in response["messages"]]

In [ ]:
response = agent_executor.invoke(
    {"messages": "¿Va a llover mañana en Madrid? Busca primero la fecha actual para dar el resultado correcto"},
    {"configurable": {"thread_id": "nueva_key"}}
)

msgs = [msg.pretty_print() for msg in response["messages"]]

### Preguntas concatenadas

In [ ]:
memory = MemorySaver()
llm = ChatOpenAI(model="gpt-4o-mini")

agent_executor = create_react_agent(llm, tools, checkpointer=memory, debug=False)

config = {"configurable": {"thread_id": "abcd1234"}}

response = agent_executor.invoke(
    {"messages": "¿Cómo se llama la mujer del actor que hizo de Eddard Stark en Juego de Tronos?"},
    config
)

msgs = [msg.pretty_print() for msg in response["messages"]]